# 闸门模型 (Gate Model)

`闸门 (Gate)` 类代表水管理系统中的一个可控闸门。它根据上游和下游的水位以及闸门的开启高度来模拟出流量。本Jupyter Notebook解释了如何使用 `Gate` 模型，并模拟其在变化的控制信号下的行为。

## 状态变量

闸门的状态由以下变量描述：

- `opening` (float): 闸门的当前开启高度 (m)。
- `outflow` (float): 通过闸门的出流量 (m³/s)。

## 参数

闸门的物理和操作特性由以下参数定义：

- `discharge_coefficient` (float): 一个无量纲系数，代表闸门流量的效率。
- `width` (float): 闸门的宽度 (m)。
- `max_opening` (float): 闸门可能的最大开启高度 (m)。
- `max_rate_of_change` (float): 闸门开启度可以变化的最大速度 (m/s)。

## 仿真示例

以下示例演示了如何创建一个 `Gate` 实例并模拟其操作。仿真显示了闸门开启到一个目标水平，然后关闭的过程。在每一步中，出流量都是基于假定的恒定水位计算的。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` (`pip install matplotlib`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.gate import Gate
from swp.core.interfaces import State, Parameters

# 闸门的初始状态和参数
initial_state = State(opening=0.0, outflow=0.0)
parameters = Parameters(
    discharge_coefficient=0.61,
    width=5.0,              # 米
    max_opening=2.0,        # 米
    max_rate_of_change=0.05 # m/s
)

# 创建一个Gate实例
gate = Gate(name="control_gate", initial_state=initial_state, parameters=parameters)

# 仿真设置
dt = 1  # 时间步长（秒）
simulation_duration = 120  # 2分钟
num_steps = int(simulation_duration / dt)

# 存储结果用于绘图
history = []

# 定义仿真的外部条件和控制信号
upstream_water_level = 10.0  # 恒定的上游水位 (m)
downstream_water_level = 8.0   # 恒定的下游水位 (m)

# 运行仿真循环
for t in range(num_steps):
    # 根据时间定义控制信号
    if t < 20:
        target_opening = 1.5  # 命令开启
    elif t > 80:
        target_opening = 0.5  # 命令部分关闭
    else:
        target_opening = 1.5 # 保持开启度
        
    action = {
        'control_signal': target_opening,
        'upstream_head': upstream_water_level,
        'downstream_head': downstream_water_level
    }
    
    current_state = gate.step(action=action, dt=dt)
    history.append(current_state.copy())

# 打印最终状态
print("最终状态:", gate.get_state())

# 准备绘图数据
time_steps = [i * dt for i in range(num_steps)]
openings = [s['opening'] for s in history]
outflows = [s['outflow'] for s in history]

# 绘制结果
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

ax1.plot(time_steps, openings, label='闸门开启度 (Gate Opening)')
ax1.set_ylabel('开启度 (m)')
ax1.set_title('闸门操作仿真')
ax1.axhline(1.5, color='r', linestyle='--', label='目标开启度 (1.5m)')
ax1.axhline(0.5, color='g', linestyle='--', label='目标开启度 (0.5m)')
ax1.grid(True)
ax1.legend()

ax2.plot(time_steps, outflows, label='出流量 (Outflow)', color='orange')
ax2.set_xlabel('时间 (秒)')
ax2.set_ylabel('出流量 (m³/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
